In [ ]:
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

# BLIP 모델 로딩 (이미지 캡션 생성 모델)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model_blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# 이미지 로딩 및 캡션 생성 함수
def generate_caption(image_path):
    # 이미지 경로 확인
    if not os.path.exists(image_path):
        print(f"파일이 존재하지 않습니다: {image_path}")
        return None
    
    try:
        # 이미지 열기
        image = Image.open(image_path)
    except Exception as e:
        print(f"이미지 열기 오류: {e}")
        return None
    
    # 이미지 전처리
    inputs = processor(images=image, return_tensors="pt")
    
    # BLIP 모델로 이미지 캡션 생성
    with torch.no_grad():
        output = model_blip.generate(**inputs)
    
    # 캡션 텍스트 반환
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

# 로컬 이미지 경로
image_path = "./data/image_1.png"  # 이미지 경로 (절대 경로로 설정해보세요)

# 이미지 캡션 생성
caption = generate_caption(image_path)

# 캡션이 정상적으로 생성된 경우에만 진행
if caption:
    # GPT-4 프롬프트 생성
    prompt_template = """
    You are an expert in recommending perfumes based on scent descriptions. Analyze the following description, find matching perfumes from Fragrantica's notes database, and provide recommendations. Additionally, give a concise yet detailed reason for each recommendation in Korean, focusing on emotional depth and vivid imagery.

    Please respond directly with the recommendations and explanations in Korean. Do not include introductory or transitional phrases.

    {input_text}

    Do not provide any additional information.
    """
    
    # Langchain을 통해 GPT-4 모델을 사용하여 향수 추천 받기
    try:
        # ChatOpenAI 객체 생성
        model_chat = ChatOpenAI(model="gpt-4o")
        
        # 프롬프트 템플릿을 Langchain PromptTemplate으로 설정
        prompt = PromptTemplate(input_variables=["input_text"], template=prompt_template)

        # 포맷된 텍스트를 생성하여 전달
        formatted_prompt = prompt.format(input_text=caption)

        # GPT-4 모델에 포맷된 프롬프트 텍스트 전달
        response = model_chat.invoke(formatted_prompt)  # 텍스트 형태로 전달
        
        # 향수 추천 출력
        print(f"향수 추천 답변 : {response}")
    
    except Exception as e:
        print(f"GPT-4 요청 중 오류 발생: {e}")
else:
    print("이미지 캡션 생성 실패")

In [13]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ------- -------------------------------- 2.9/16.1 MB 16.7 MB/s eta 0:00:01
   ---------------------------------- ----- 13.9/16.1 MB 37.9 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 30.6 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import mysql.connector

# 1. 페이지 요청 및 데이터 추출
url = "https://www.bysuco.com/product/show/10155"  # 실제 URL로 변경
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# descWrap 클래스를 가진 div에서 텍스트 가져오기
desc_wrap = soup.find("div", class_="descWrap")
brand = soup.find("a", class_="tit")
title = soup.find("h3", class_="desc ellipsisTwo")

if desc_wrap:
    text_content = desc_wrap.get_text(separator=" ", strip=True)  # 텍스트 추출 및 공백 제거
    # print(text_content)
else:
    print("descWrap 요소를 찾을 수 없습니다.")

if brand:
    brand_name = brand.get_text(separator=" ", strip=True)
    print(brand_name)
else:
    print("브랜드 이름을 찾을 수 없습니다")

# 필요한 데이터 추출
brand = grade = top_note = middle_note = base_note = description = None  # 모든 변수 초기화

# 텍스트에서 각 항목을 정규 표현식으로 추출
brand_match = re.search(r"([^\n]+)", brand_name)
if brand_match:
    brand = brand_match.group(1).strip()
    print("향수 브랜드:", brand)

grade_match = re.search(r"\[부향률\]\s*-\s*([^\n]+)", text_content)
if grade_match:
    grade = grade_match.group(1).strip()
    print("부향률:", grade)

top_note_match = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", text_content)
if top_note_match:
    top_note = top_note_match.group(1).strip()
    print("탑 노트:", top_note)

middle_note_match = re.search(r"미들 노트:([^\n]+)", text_content)
if middle_note_match:
    middle_note = middle_note_match.group(1).strip()
    print("미들 노트:", middle_note)

base_note_match = re.search(r"베이스 노트:([^\n]+)", text_content)
if base_note_match:
    base_note = base_note_match.group(1).strip()
    print("베이스 노트:", base_note)

description_match = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", text_content)
if description_match:
    description = description_match.group(1).strip()
    print("향 설명:", description)

# 2. MySQL 데이터베이스 연결
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',          # 본인의 MySQL 사용자명
    password='ohgiraffers',      # 본인의 MySQL 비밀번호
    database='test_db',          # 데이터베이스 이름
    charset='utf8mb4'
)

try:
    with connection.cursor() as cursor:
        # 테이블 생성 (존재하지 않는 경우)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS top_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                top_note TEXT
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS middle_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                middle_note TEXT
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS base_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                base_note TEXT
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS perfume (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(50),
                brand VARCHAR(50),
                grade VARCHAR(50),
                description TEXT,
                top_note TEXT,
                middle_note TEXT,
                base_note TEXT
                
            )
        ''')
        
        
        # 3. 데이터 삽입
        # sql = "INSERT INTO perfume_data (concentration, top_note, middle_note, base_note, description) VALUES (%s, %s, %s, %s, %s, %s)"
        # cursor.execute(sql, (concentration, top_note, middle_note, base_note, description))
        
        # 3. 데이터 삽입
        # perfume_data 테이블에 데이터 삽입
        cursor.execute("INSERT INTO perfume_data (concentration, description) VALUES ( %s, %s)", (concentration, description))
        id = cursor.lastrowid  # 방금 삽입한 perfume_data의 ID를 가져옵니다
        
        # 각 노트 데이터를 삽입 (top_note, middle_note, base_note)
        if top_note:
            cursor.execute("INSERT INTO top_note (id, note) VALUES (%s, %s)", (id, top_note))
        
        if middle_note:
            cursor.execute("INSERT INTO middle_note (id, note) VALUES (%s, %s)", (id, middle_note))
        
        if base_note:
            cursor.execute("INSERT INTO base_note (id, note) VALUES (%s, %s)", (id, base_note))
    
    # 변경 사항 저장
    connection.commit()

finally:
    # 연결 종료
    connection.close()

print("데이터가 성공적으로 저장되었습니다!")


딥티크
향수 브랜드: 딥티크
부향률: 오 드 퍼퓸
탑 노트: 알데하이드, 핑크 페퍼, 앤젤리카, 베르가못
미들 노트: 아이리스, 터키쉬 장미
베이스 노트: 머스크, 암브레트, 당근, 앰버그리스, 샌달우드, 레더, 앰버우드
향 설명: 달콤한 숨소리로 로맨틱한 포옹을 하는 듯한 관능적인 체취를 느낄 수 있는 향


ProgrammingError: 1054 (42S22): Unknown column 'note' in 'field list'

In [11]:
import requests
from bs4 import BeautifulSoup
import re
import mysql.connector

# 1. 페이지 요청 및 데이터 추출
url = "https://www.bysuco.com/product/show/10155"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 데이터 추출
desc_wrap = soup.find("div", class_="descWrap")
brand = soup.find("a", class_="tit")
title = soup.find("h3", class_="desc ellipsisTwo")

if desc_wrap:
    text_content = desc_wrap.get_text(separator=" ", strip=True)
else:
    print("descWrap 요소를 찾을 수 없습니다.")

brand_name = brand.get_text(separator=" ", strip=True) if brand else "Unknown"
title = title.get_text(separator=" ", strip=True) if title else "Unknown"

# 텍스트에서 각 항목 추출
concentration = re.search(r"\[부향률\]\s*-\s*([^\n]+)", text_content)
concentration = concentration.group(1).strip() if concentration else "Unknown"

top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", text_content)
top_note = top_note.group(1).strip() if top_note else "Unknown"

middle_note = re.search(r"미들 노트:([^\n]+)", text_content)
middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

base_note = re.search(r"베이스 노트:([^\n]+)", text_content)
base_note = base_note.group(1).strip() if base_note else "Unknown"

description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", text_content)
description = description.group(1).strip() if description else "Unknown"

# 2. MySQL 연결
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

try:
    with connection.cursor() as cursor:
        # 테이블 생성
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS top_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                top_note TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS middle_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                middle_note TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS base_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                base_note TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS perfume (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(50),
                brand VARCHAR(50),
                grade VARCHAR(50),
                description TEXT,
                top_note TEXT,
                middle_note TEXT,
                base_note TEXT
            )
        ''')

        # 데이터 삽입
        cursor.execute("INSERT INTO perfume (name, brand, grade, description, top_note, middle_note, base_note) VALUES (%s, %s, %s, %s, %s, %s, %s)", 
                    (title, brand_name, concentration, description, top_note, middle_note, base_note))
        perfume_id = cursor.lastrowid

        # 각 노트 삽입
        if top_note:
            cursor.execute("INSERT INTO top_note (id, top_note) VALUES (%s, %s)", (perfume_id, top_note))
        if middle_note:
            cursor.execute("INSERT INTO middle_note (id, middle_note) VALUES (%s, %s)", (perfume_id, middle_note))
        if base_note:
            cursor.execute("INSERT INTO base_note (id, base_note) VALUES (%s, %s)", (perfume_id, base_note))

        # 변경 사항 저장
        connection.commit()

finally:
    connection.close()

print("데이터가 성공적으로 저장되었습니다!")


데이터가 성공적으로 저장되었습니다!


In [20]:
!pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.bysuco.com/product?num=60&page=1&orderBy=popular&keyword=&kind=bt')
time.sleep(3)

# 상품 목록에서 상세 페이지 링크 추출
product_links = []
elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
for element in elements:
    href = element.get_attribute('href')
    product_links.append(href)

# 각 상품 상세 페이지에서 텍스트 추출
for link in product_links:
    driver.get(link)
    time.sleep(3)

    try:
        # 클래스 textWrap, ellipsisTwo, desc, tit, descWrap 텍스트 추출
        text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
        ellipsis_two = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
        desc = driver.find_element(By.CLASS_NAME, 'desc').text
        tit = driver.find_element(By.CLASS_NAME, 'tit').text
        desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text

        print(f"상품 텍스트: {text_wrap}")
        print(f"ellipsisTwo: {ellipsis_two}")
        print(f"상품 브랜드: {tit}")
        print(f"descWrap: {desc_wrap}")
    except Exception as e:
        print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")

# 드라이버 종료
driver.quit()

상품 텍스트: 
ellipsisTwo: 오드 우드 오 드 퍼퓸
상품 브랜드: 톰 포드
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 우디 / 오우드 / 웜 스파이시

[메인 노트]
- 싱글 노트: 아가우드, 브라질리안 로즈우드, 샌달우드, 카다멈, 바닐라, 사천 페퍼, 베티버, 통카 빈, 앰버

[향 설명]
- 청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
상품 텍스트: 
ellipsisTwo: 화이트 스웨이드 오 드 퍼퓸
상품 브랜드: 톰 포드
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 머스크 / 파우더리 / 레더

[메인 노트]
- 탑 노트: 타임, 티
- 미들 노트: 릴리오브더밸리, 샤프론, 로즈
- 베이스 노트: 스웨이드, 머스크, 샌달우드, 올리바넘, 앰버

[향 설명]
- 가죽의 향이 섞인 머스크가 표현하는 제 2의 피부의 향
상품 텍스트: 
ellipsisTwo: 플레르 드 뽀 오 드 퍼퓸
상품 브랜드: 딥티크
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 머스크 / 파우더리 / 아이리스

[메인 노트]
- 탑 노트: 알데하이드, 핑크 페퍼, 앤젤리카, 베르가못
- 미들 노트: 아이리스, 터키쉬 장미
- 베이스 노트: 머스크, 암브레트, 당근, 앰버그리스, 샌달우드, 레더, 앰버우드

[향 설명]
- 달콤한 숨소리로 로맨틱한 포옹을 하는 듯한 관능적인 체취를 느낄 수 있는 향
상품 텍스트: 
ellipsisTwo: 테싯 오 드 퍼퓸
상품 브랜드: 이솝
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 시트러스 / 아로마틱 / 프레쉬 스파이시

[메인 노트]
- 탑 노트: 유자, 시트러스
- 미들 노트: 바질
- 베이스 노트: 베티버, 클로브

[향 설명]
- 이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향
상품 텍스트: 
ellipsisTwo: 오르페옹 오 드 퍼퓸
상품 브랜드: 딥티크
descWrap: [부향률] 
-

KeyboardInterrupt: 

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import mysql.connector
import re
import time

# MySQL 연결 설정
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.bysuco.com/product?num=60&page=1&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')

# 상품 목록에서 상세 페이지 링크 추출
product_links = []
elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
for element in elements:
    href = element.get_attribute('href')
    product_links.append(href)

# 각 상품 상세 페이지에서 데이터 추출 및 MySQL 삽입
for link in product_links:
    driver.get(link)

    try:
        # 페이지가 완전히 로드될 때까지 기다림
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap')))

        # 필요한 데이터 추출 (상품명, 브랜드, 부향률, 노트, 설명 등)
        text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
        title = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
        brand_name = driver.find_element(By.CLASS_NAME, 'tit').text
        desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text

        # 텍스트에서 각 항목 추출
        grade = re.search(r"\[부향률\]\s*-\s*([^\n]+)", desc_wrap)
        grade = grade.group(1).strip() if grade else "Unknown"

        top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
        top_note = top_note.group(1).strip() if top_note else "Unknown"

        middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
        middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

        base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
        base_note = base_note.group(1).strip() if base_note else "Unknown"
        
        single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
        single_note = single_note.group(1).strip() if single_note else "Unknown"

        description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
        description = description.group(1).strip() if description else "Unknown"

        # MySQL 데이터 삽입
        with connection.cursor() as cursor:
            # 데이터베이스에 테이블이 없으면 생성
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS top_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                top_note TEXT
            )
            ''')
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS middle_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                middle_note TEXT
            )
            ''')
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS base_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                base_note TEXT
            )
            ''')
            
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS single_note (
            id INT AUTO_INCREMENT PRIMARY KEY,
            single_note TEXT
            )
            ''')
            
            cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(50),
                brand VARCHAR(50),
                grade TEXT,
                description TEXT,
                top_note TEXT,
                middle_note TEXT,
                base_note TEXT,
                single_note TEXT
            )''')

            # 데이터 삽입
            cursor.execute("INSERT INTO perfume (name, brand, grade, description, top_note, middle_note, base_note, single_note) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                        (title, brand_name, grade, description, top_note, middle_note, base_note, single_note))
            perfume_id = cursor.lastrowid

            # 각 노트 삽입
            if top_note:
                cursor.execute("INSERT INTO top_note (id, top_note) VALUES (%s, %s)", (perfume_id, top_note))
            if middle_note:
                cursor.execute("INSERT INTO middle_note (id, middle_note) VALUES (%s, %s)", (perfume_id, middle_note))
            if base_note:
                cursor.execute("INSERT INTO base_note (id, base_note) VALUES (%s, %s)", (perfume_id, base_note))
            if single_note:
                cursor.execute("INSERT INTO single_note (id, single_note) VALUES (%s, %s)", (perfume_id, single_note))

            # 변경 사항 저장
            connection.commit()

    except Exception as e:
        print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")
        continue  # 에러가 발생하면 다음 상품으로 넘어감

# 드라이버 종료 및 데이터베이스 연결 종료
driver.quit()
connection.close()

print("모든 데이터가 성공적으로 저장되었습니다!")


모든 데이터가 성공적으로 저장되었습니다!
